In [1]:
### imports

# external modules
import os
import sys
import time
import numpy as np
import matplotlib.pyplot as plt
import importlib
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model, Sequential, load_model
import importlib
from sklearn.preprocessing import StandardScaler

# local modules
sys.path.append('../utils')
import csv_utils as csvu
import json_utils as jsonu
import dataframe_utils as dfu
import hist_utils as hu
import autoencoder_utils as aeu
import plot_utils as pu
import generate_data_utils as gdu
import refruns_utils as rru
importlib.reload(csvu)
importlib.reload(jsonu)
importlib.reload(dfu)
importlib.reload(hu)
importlib.reload(aeu)
importlib.reload(pu)
importlib.reload(gdu)
importlib.reload(rru)
sys.path.append('../src')
sys.path.append('../src/classifiers')
sys.path.append('../src/cloudfitters')
import HistStruct
importlib.reload(HistStruct)
import SubHistStruct
importlib.reload(SubHistStruct)
import FlexiStruct
importlib.reload(FlexiStruct)
import DataLoader
importlib.reload(DataLoader)
import AutoEncoder
importlib.reload(AutoEncoder)
import SeminormalFitter
import GaussianKdeFitter
import HyperRectangleFitter
importlib.reload(SeminormalFitter)
importlib.reload(GaussianKdeFitter)
importlib.reload(HyperRectangleFitter)

2022-07-28 19:37:43.451423: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cvmfs/sft.cern.ch/lcg/releases/MCGenerators/thepeg/2.2.3-88592/x86_64-centos7-gcc11-opt/lib/ThePEG:/cvmfs/sft.cern.ch/lcg/releases/MCGenerators/herwig++/7.2.3-35f7a/x86_64-centos7-gcc11-opt/lib/Herwig:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/jaxlib/mlir/_mlir_libs:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/torch/lib:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/onnxruntime/capi/:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/tensorflow:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/tensorflow/contrib

<module 'HyperRectangleFitter' from '/eos/home-i01/k/khowey/SWAN_projects/ML4DQMDC-PixelAE/Merging Histogram Notebooks/../src/cloudfitters/HyperRectangleFitter.py'>

In [2]:
### Speed Controls and Run Mode

# Disables all plots for large datasets where speed is more important
createPlots = False

In [3]:
### Evaluation Parameters

# Select the bias towards recall against precision, treated as a factor (so < 1 biases towards precision, 1 is equal importance, and > 1 biases towards recall)
wpBiasFactor = 20
fmBiasFactor = 2

In [4]:
### Defining bad runs
badruns = {'2017B':
                [
                    297048,
                    297282,
                    297283,
                    297284,
                    297287,
                    297288,
                    297289,
                    299316,
                    299317,
                    299318,
                    299324,
                    299326,
                    301086,
                    301086,
                    303948,
                    297047, #close but, true bad for all 8
                    297169, #true bad for all 8
                    297211, #Reconstructs well
                    299325, #Reconstructs well
                    297664, #true bad for all 8
                    299317, #true bad for all 8
                    297169, #true bad for all 8
                    297502
                ],
             '2017C':[
                  300781, # bad for tracking (pixels were excluded.
                  300079, # is bad for strips and then also for tracking
                  302029, # Poor detector elements for strips - Should be mildly anomalous, but technically good 
                  300576, # Poor detector elements for strips - Should be mildly anomalous, but technically good
                  300574, # Poor detector elements for strips - Should be mildly anomalous, but technically good
                  300282, # Poor detector elements for strips - Should be mildly anomalous, but technically good
                  301912, # Half bad for pixels (lost HV or readout card)  
                  301086, # Half bad for pixels (lost HV or readout card)  
                  300283, # Half bad for pixels (lost HV or readout card) 
                  300282, # Half bad for pixels (lost HV or readout card) 
                  300281, # Half bad for pixels (lost HV or readout card) 
                  300239, # Half bad for pixels (lost HV or readout card)
                  301394, # Marginal for pixels
                  301183, # Marginal for pixels
                  300398, # Marginal for pixels
                  300389, # Marginal for pixels
                  300365  # Marginal for pixels
             ],
             '2017E':[
                 304740, # Bad for pixels and tracking - holes in PXLayer 1
                 304776, # Bad for pixels and tracking - holes in PXLayer 1
                 304506, # Portcard problem for pixels
                 304507, # Portcard problem for pixels 
                 303989, # Bad for pixels, power supply died
                 303824  # Partly bad for strips due to a test
             ],
             '2017F':[
                 306422, # Partly bad for strips - 2 data readouts failed 
                 306423, # Partly bad for strips - 2 data readouts failed
                 306425, # Partly bad for strips - 2 data readouts failed
                 305440, # Partly bad for strips - 1 data readout failed
                 305441, # Partly bad for strips - 1 data readout failed
                 305249, # Bad for pixels - half of disk failed 
                 305250, # Bad for pixels - half of disk failed
                 305064, # Marginal for pixels - some readout failed
             ],
            '2018': # needs to be re-checked, not guaranteed to be fully correct or representative.
                [
                317479,
                317480,
                317481,
                317482,
                319847
                ]}


In [5]:
### Select a reference run and get data
rundict = jsonu.loadjson('../jsons/CertHelperRefRuns.json')

# Select any run numbers to get a training set from that run's reference.
runNums = [303824, 306422]
refRuns = []
eras = []
years = []
dataDict = {}
badrunsls = {}
trainrunsls = {}
goodrunsls = {}
for runNum in runNums:
    runls = {}
    for run in rundict:
        if run['run_number'] == runNum:
            runls.update(run)
    if runls == {}:
        raise Exception('Run not found - ' + str(runNum))
    
    year = runls['dataset'][11:15]
    if year not in years: years.append(year)
    era = runls['dataset'][15]
    if era not in eras: eras.append(era)
    ref_run = runls['reference_run_number']
    
    # Don't need duplicates
    if ref_run in refRuns:
        continue
    refRuns.append(ref_run)
    
    # Get the runs associated with found reference
    outputRuns = {}
    outputBad = {}
    for run in rundict:
        tempRef = run['reference_run_number']
        if tempRef == ref_run:
            runls = {}
            runls[str(run['run_number'])] = [[-1]]
            if run['run_number'] in badruns[year+era]:
                print('Found bad run :' + str(run))
                outputBad.update(runls)
            else:
                outputRuns.update(runls)
    
    # Perform structuring for compatibility with autoencoders
    dataDict[year + era] = outputRuns
    badrunsls[year + era] = outputBad
    trainrunsls[year + era] = {}
    goodrunsls[year + era] = {}
    
    # Select training and testing set
    for i,run in enumerate(dataDict[year + era]):
        if i > 5 and i < 11:
            goodrunsls[year + era][str(run)] = [[-1]]
        else:
            trainrunsls[year + era][str(run)] = [[-1]]

if len(years) != 1: raise Exception('Year of length 0 or >1 unimplemented!')

Found bad run :{'run_number': 303824, 'run_reconstruction_type': 'rerecoul', 'reference_run_number': 304158, 'reference_run_reconstruction_type': 'rerecoul', 'dataset': '/ReReco/Run2017E_UL2019/DQM'}
Found bad run :{'run_number': 303989, 'run_reconstruction_type': 'rerecoul', 'reference_run_number': 304158, 'reference_run_reconstruction_type': 'rerecoul', 'dataset': '/ReReco/Run2017E_UL2019/DQM'}
Found bad run :{'run_number': 304740, 'run_reconstruction_type': 'rerecoul', 'reference_run_number': 304158, 'reference_run_reconstruction_type': 'rerecoul', 'dataset': '/ReReco/Run2017E_UL2019/DQM'}
Found bad run :{'run_number': 305250, 'run_reconstruction_type': 'rerecoul', 'reference_run_number': 306459, 'reference_run_reconstruction_type': 'rerecoul', 'dataset': '/ReReco/Run2017F_UL2019/DQM'}
Found bad run :{'run_number': 306422, 'run_reconstruction_type': 'rerecoul', 'reference_run_number': 306459, 'reference_run_reconstruction_type': 'rerecoul', 'dataset': '/ReReco/Run2017F_UL2019/DQM'}


In [6]:
### Data Controls and Selection - 1D Autoncoder

# The directory data is located in
datadir = {}
for era in eras:
    datadir[year + era] = '../data/' + year + era + '/'

# Create a list of histograms to include
# Pair histograms to be combined on the same line
histnames = [['chargeInner_PXLayer_1', 'chargeInner_PXLayer_2', 'chargeInner_PXLayer_3', 'chargeInner_PXLayer_4', 'chargeOuter_PXLayer_1', 'chargeOuter_PXLayer_2', 'chargeOuter_PXLayer_3', 'chargeOuter_PXLayer_4', 'adc_PXLayer_1', 'adc_PXLayer_2', 'adc_PXLayer_3', 'adc_PXLayer_4', 'size_PXLayer_1', 'size_PXLayer_2', 'size_PXLayer_3', 'size_PXLayer_4'], ['charge_PXDisk_+1', 'charge_PXDisk_+2', 'charge_PXDisk_+3', 'adc_PXDisk_+1', 'adc_PXDisk_+2', 'adc_PXDisk_+3', 'size_PXDisk_+1', 'size_PXDisk_+2', 'size_PXDisk_+3'], ['charge_PXDisk_-1', 'charge_PXDisk_-2', 'charge_PXDisk_-3', 'adc_PXDisk_-1', 'adc_PXDisk_-2', 'adc_PXDisk_-3', 'size_PXDisk_-1', 'size_PXDisk_-2', 'size_PXDisk_-3'], ['NormalizedHitResiduals_TIB__Layer__1', 'NormalizedHitResiduals_TIB__Layer__2', 'NormalizedHitResiduals_TIB__Layer__3', 'NormalizedHitResiduals_TIB__Layer__4', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__1', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__2', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__3', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__4'], ['NormalizedHitResiduals_TOB__Layer__1', 'NormalizedHitResiduals_TOB__Layer__2', 'NormalizedHitResiduals_TOB__Layer__3', 'NormalizedHitResiduals_TOB__Layer__4', 'NormalizedHitResiduals_TOB__Layer__5', 'NormalizedHitResiduals_TOB__Layer__6', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__1', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__2', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__3', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__4', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__5', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__6'], ['Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__1', 'Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__2', 'Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__3'], ['NormalizedHitResiduals_TID__wheel__1', 'NormalizedHitResiduals_TID__wheel__2', 'NormalizedHitResiduals_TID__wheel__3', 'Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__1', 'Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__2', 'Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__3'], ['Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__1', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__2', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__3', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__4', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__5', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__6', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__7', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__8', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__9'], ['NormalizedHitResiduals_TEC__wheel__1', 'NormalizedHitResiduals_TEC__wheel__2', 'NormalizedHitResiduals_TEC__wheel__3', 'NormalizedHitResiduals_TEC__wheel__4', 'NormalizedHitResiduals_TEC__wheel__5', 'NormalizedHitResiduals_TEC__wheel__6', 'NormalizedHitResiduals_TEC__wheel__7', 'NormalizedHitResiduals_TEC__wheel__8', 'NormalizedHitResiduals_TEC__wheel__9', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__1', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__2', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__3', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__4', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__5', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__6', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__7', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__8', 'Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__9'], ['NumberOfRecHitsPerTrack_lumiFlag_GenTk', 'Chi2oNDF_lumiFlag_GenTk', 'goodvtxNbr'], ['num_clusters_ontrack_PXBarrel', 'num_clusters_ontrack_PXForward']]

# Read new data or use previously saved data & should data be saved
readnew = True
save = False

In [7]:
### Define Run Properties - Combined Autoencoder
# in this cell all major run properties are going to be set,

# Set whether to train globally or locally or in a development/testing mode
training_mode = 'development'

In [8]:
### Model Controls and Selection - 1D Autoencoder

plotNames = 'Test'
name = plotNames+'plots'

# Choose whether to train a new model or load one
trainnew = True
savemodel = True
modelname = plotNames

In [9]:
### Define Training Mode Parameters - Combined Autoencoder
if training_mode == 'global':
    runsls_training = None # use none to not add a mask for training (can e.g. use DCS-bit on mask)
    runsls_good = None # use none to not add a mask for good runs (can e.g. use averages of training set)
    runsls_bad = badrunsls[year] # predefined bad runs
    print('selected runs/lumisections for training: all')
    
elif training_mode == 'local':
    # train locally on a small set of runs
    # - either on n runs preceding a chosen application run,
    # - or on the run associated as reference to the chosen application run.
    # - this only works for a single era
    
    available_runs = dfu.get_runs( dfu.select_dcson( csvu.read_csv('../data/DF'+year+era+'_'+histnames[0][0]+'.csv') ) )
    # Cherry picked really bad run
    run_application = 299316
    #run_application = 299317
    run_application_index = available_runs.index(run_application)
    # select training set
    usereference = False
    if usereference:
        run_reference = rru.get_reference_run( run_application, jsonfile='../utils/json_allRunsRefRuns.json' )
        if run_reference<0:
            raise Exception('no valid reference run has been defined for run {}'.format(run_application))
        runsls_training = jsonu.tuplelist_to_jsondict([(run_reference,[-1])])
    else:
        ntraining = 5
        offset = 0 # normal case: offset = 0 (just use 5 previous runs)
        
        # Selects the 5 previous runs for training
        runsls_training = jsonu.tuplelist_to_jsondict([(el,[-1]) for el in available_runs[run_application_index-ntraining-offset:run_application_index-offset]])
    #runsls_bad = badrunsls[year]
    #runsls_good = jsonu.tuplelist_to_jsondict([(run_application,[-1])])
    runsls_bad = jsonu.tuplelist_to_jsondict([(run_application,[-1])])
    runsls_good = runsls_training
    print('selected runs/lumisections for training: ')
    print(runsls_training)
    print('selected runs/lumisections as good test set:')
    print(runsls_good)
    print('selected runs/lumisections as bad test set:')
    print(runsls_bad)
        
elif training_mode == 'development':
    # train on a user-defined subset of runs
    
   # Select runs to be used in training from the user-defined list
    runsls_training = {}
    runsls_bad = {}
    runsls_good = {}
    for era in eras:
        runsls_training.update(trainrunsls[year + era])
        # Select bad runs to test on in the user-defined list
        runsls_bad.update(badrunsls[year + era])
        # Select good runs to test on in the user-defined list
        runsls_good.update(goodrunsls[year + era])
    
    print('selected runs/lumisections for training: ')
    print(runsls_training)
    print('selected runs/lumisections as good test set:')
    print(runsls_good)
    print('selected runs/lumisections as bad test set:')
    print(runsls_bad)

selected runs/lumisections for training: 
{'303819': [[-1]], '303999': [[-1]], '304119': [[-1]], '304120': [[-1]], '304197': [[-1]], '304505': [[-1]], '304449': [[-1]], '304452': [[-1]], '304508': [[-1]], '304625': [[-1]], '304655': [[-1]], '304737': [[-1]], '304778': [[-1]], '306459': [[-1]], '304196': [[-1]], '305310': [[-1]], '305040': [[-1]], '305043': [[-1]], '305185': [[-1]], '305204': [[-1]], '305234': [[-1]], '305376': [[-1]], '306042': [[-1]], '306051': [[-1]], '305406': [[-1]], '306122': [[-1]], '306134': [[-1]], '306137': [[-1]], '306154': [[-1]], '306170': [[-1]], '306417': [[-1]], '306432': [[-1]], '306456': [[-1]], '305516': [[-1]], '305586': [[-1]], '305588': [[-1]], '305590': [[-1]], '305809': [[-1]], '305832': [[-1]], '305840': [[-1]], '305898': [[-1]], '306029': [[-1]], '306037': [[-1]], '306095': [[-1]]}
selected runs/lumisections as good test set:
{'304198': [[-1]], '304199': [[-1]], '304209': [[-1]], '304333': [[-1]], '304446': [[-1]], '305247': [[-1]], '305313': [

In [10]:
### Data Import

# Create a new HistStruct from the data
if readnew:
    # Initializations
    dloader = DataLoader.DataLoader()
    histstruct = FlexiStruct.FlexiStruct()
    histstruct.reset_histlist(histnames)
    
    # Unpack histnames and add every histogram individually
    for era in eras:
        for histnamegroup in histnames:
            for histname in histnamegroup:
                print('Adding {}...'.format(histname))
                
                # Bring the histograms into memory from storage for later use
                filename = datadir[year+era] + 'DF' + year + era + '_' + histname + '.csv'
                df = dloader.get_dataframe_from_file( filename )
                
                # In case of local training, we can remove most of the histograms
                if( runsls_training is not None and runsls_good is not None and runsls_bad is not None ):
                    runsls_total = {k: v for d in (runsls_training, runsls_good, runsls_bad) for k, v in d.items()}
                    df = dfu.select_runsls( df, runsls_total )    
                
                df = dfu.rm_duplicates(df)
                # Store the data in the histstruct object managing this whole thing
                histstruct.add_dataframe( df, rebinningfactor = 1, standardbincount = 102 )
        print('Found {} histograms'.format(len(histstruct.runnbs)))

# Load a previously saved HistStruct
else:
    # Load histstruct from storage
    histstruct = SubHistStruct.SubHistStruct.load( 'histstruct_global_20220201.zip', verbose=False )
    nbadruns = len([name for name in list(histstruct.masks.keys()) if ('bad' in name and name!='bad')])
    
    print('loaded a histstruct with the following properties:')
    print(histstruct)
    # Count of bad runs, presumably for later use
    nbadruns = len([name for name in list(histstruct.masks.keys()) if 'bad' in name])
    
print('Created a histstruct with the following properties:')
print('- number of histogram types: {}'.format(len(histstruct.histnames)))
print('- number of lumisections: {}'.format(len(histstruct.lsnbs)))

Adding chargeInner_PXLayer_1...
Adding chargeInner_PXLayer_2...
Adding chargeInner_PXLayer_3...
Adding chargeInner_PXLayer_4...
Adding chargeOuter_PXLayer_1...
Adding chargeOuter_PXLayer_2...
Adding chargeOuter_PXLayer_3...
Adding chargeOuter_PXLayer_4...
Adding adc_PXLayer_1...
Adding adc_PXLayer_2...
Adding adc_PXLayer_3...
Adding adc_PXLayer_4...
Adding size_PXLayer_1...
Adding size_PXLayer_2...
Adding size_PXLayer_3...
Adding size_PXLayer_4...
Adding charge_PXDisk_+1...
Adding charge_PXDisk_+2...
Adding charge_PXDisk_+3...
Adding adc_PXDisk_+1...
Adding adc_PXDisk_+2...
Adding adc_PXDisk_+3...
Adding size_PXDisk_+1...
Adding size_PXDisk_+2...
Adding size_PXDisk_+3...
Adding charge_PXDisk_-1...
Adding charge_PXDisk_-2...
Adding charge_PXDisk_-3...
Adding adc_PXDisk_-1...
Adding adc_PXDisk_-2...
Adding adc_PXDisk_-3...
Adding size_PXDisk_-1...
Adding size_PXDisk_-2...
Adding size_PXDisk_-3...
Adding NormalizedHitResiduals_TIB__Layer__1...
Adding NormalizedHitResiduals_TIB__Layer__2..

Adding adc_PXDisk_-1...
Adding adc_PXDisk_-2...
Adding adc_PXDisk_-3...
Adding size_PXDisk_-1...
Adding size_PXDisk_-2...
Adding size_PXDisk_-3...
Adding NormalizedHitResiduals_TIB__Layer__1...
Adding NormalizedHitResiduals_TIB__Layer__2...
Adding NormalizedHitResiduals_TIB__Layer__3...
Adding NormalizedHitResiduals_TIB__Layer__4...
Adding Summary_ClusterStoNCorr__OnTrack__TIB__layer__1...
Adding Summary_ClusterStoNCorr__OnTrack__TIB__layer__2...
Adding Summary_ClusterStoNCorr__OnTrack__TIB__layer__3...
Adding Summary_ClusterStoNCorr__OnTrack__TIB__layer__4...
Adding NormalizedHitResiduals_TOB__Layer__1...
Adding NormalizedHitResiduals_TOB__Layer__2...
Adding NormalizedHitResiduals_TOB__Layer__3...
Adding NormalizedHitResiduals_TOB__Layer__4...
Adding NormalizedHitResiduals_TOB__Layer__5...
Adding NormalizedHitResiduals_TOB__Layer__6...
Adding Summary_ClusterStoNCorr__OnTrack__TOB__layer__1...
Adding Summary_ClusterStoNCorr__OnTrack__TOB__layer__2...
Adding Summary_ClusterStoNCorr__OnT

Adding NormalizedHitResiduals_TEC__wheel__8...
Adding NormalizedHitResiduals_TEC__wheel__9...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__1...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__2...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__3...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__4...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__5...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__6...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__7...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__8...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__9...
Adding NumberOfRecHitsPerTrack_lumiFlag_GenTk...
Adding Chi2oNDF_lumiFlag_GenTk...
Adding goodvtxNbr...
Adding num_clusters_ontrack_PXBarrel...
Adding num_clusters_ontrack_PXForward...
Found 25676 histograms
Created a histstruct with the following properties:
- number of histogram types: 95
- number of lumisections: 256

In [11]:
### Add Masks to Data

if readnew:
    histstruct.add_dcsonjson_mask( 'dcson' )
    histstruct.add_goldenjson_mask('golden' )
    histstruct.add_highstat_mask( 'highstat', entries_to_bins_ratio=0 )
    histstruct.add_stat_mask( 'lowstat', max_entries_to_bins_ratio=0 )
    if runsls_training is not None: histstruct.add_json_mask( 'training', runsls_training )
    if runsls_good is not None: histstruct.add_json_mask( 'good', runsls_good )
        
    # Distinguishing bad runs
    nbadruns = 0
    if runsls_bad is not None:
        print(runsls_bad)
        histstruct.add_json_mask( 'bad', runsls_bad )
        
        # Special case for bad runs: add a mask per run (different bad runs have different characteristics)
        nbadruns = len(runsls_bad.keys())
        for i,badrun in enumerate(runsls_bad.keys()):
            histstruct.add_json_mask( 'bad{}'.format(i), {badrun:runsls_bad[badrun]} )
            
    if save:
        histstruct.save('test.pk1')
print('Assigned masks: {}'.format(list(histstruct.masks.keys())))

{'303824': [[-1]], '303989': [[-1]], '304740': [[-1]], '305250': [[-1]], '306422': [[-1]], '305249': [[-1]]}
Assigned masks: ['dcson', 'golden', 'highstat', 'lowstat', 'training', 'good', 'bad', 'bad0', 'bad1', 'bad2', 'bad3', 'bad4', 'bad5']


In [12]:
### Plotting the input data for analysis

if((training_mode=='local' or training_mode == 'development') and createPlots):

    # training and application runs
    histstruct.plot_histograms( masknames=[['dcson','highstat','training'],['dcson','highstat','good']],
                                labellist = ['training','testing'],
                                colorlist = ['blue','green']
                              )
    
    # application run and bad test runs
    histstruct.plot_histograms( masknames=[['dcson','highstat','good'],['dcson','highstat','bad0']],
                                labellist = ['good','bad'],
                                colorlist = ['green','red']
                              )
    
elif( training_mode=='global' and createPlots):
    
    # bad test runs
    for i in [0,1,2,3,4,5,6]:
        histstruct.plot_histograms( masknames=[['dcson','highstat','good'],['dcson','highstat','bad{}'.format(i)]],
                                labellist = ['typical good histograms','bad'],
                                colorlist = ['blue','red'],
                                transparencylist = [0.01,1.]
                                    )

In [13]:
def define_concatamash_autoencoder(histstruct):
    
    histslist = []
    vallist = []
    autoencoders = []
    if trainnew:
        for i,histnamegroup in enumerate(histnames):
            
            train_normhist = np.array([hu.normalize(histstruct.get_histograms(
                histname = hname, masknames = ['dcson','highstat', 'training']), 
                                                 norm="l1", axis=1) 
                                       for hname in histnamegroup]).transpose((1,0,2))
            X_train, X_val = train_test_split(train_normhist, test_size=0.4, random_state=42)
            
            # Half the total bin count
            arch = 51 * len(histnamegroup)
            
            ## Model parameters
            input_dim = X_train.shape[2] #num of predictor variables
            Input_layers=[Input(shape=input_dim) for i in range((X_train.shape[1]))]
            
            # Defining layers
            conc_layer = Concatenate()(Input_layers)
            encoder = Dense(arch * 2, activation="tanh")(conc_layer)
            encoder = Dense(arch/2, activation='relu')(encoder)
            encoder = Dense(arch/8, activation='relu')(encoder)
            encoder = Dense(arch/16, activation='relu')(encoder)
            decoder = Dense(arch/8, activation="relu")(encoder)
            decoder = Dense(arch/2, activation='relu')(encoder)
            decoder = Dense(arch * 2, activation="tanh")(decoder)
            
            Output_layers=[Dense(input_dim, activation="tanh")(decoder) for i in range(X_train.shape[1])]

            autoencoder = Model(inputs=Input_layers, outputs=Output_layers)
            autoencoders.append(autoencoder)
            
            histslist.append(X_train)
            vallist.append(X_val)
     
    # Return the histograms stored 2-Dimensionally and the autoencoders corresponding
    return(histslist, vallist, autoencoders, train_normhist)

In [14]:
(histslist, vallist, autoencoders, train_normhist) = define_concatamash_autoencoder(histstruct)

2022-07-28 19:44:46.152845: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cvmfs/sft.cern.ch/lcg/releases/MCGenerators/thepeg/2.2.3-88592/x86_64-centos7-gcc11-opt/lib/ThePEG:/cvmfs/sft.cern.ch/lcg/releases/MCGenerators/herwig++/7.2.3-35f7a/x86_64-centos7-gcc11-opt/lib/Herwig:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/jaxlib/mlir/_mlir_libs:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/torch/lib:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/onnxruntime/capi/:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/tensorflow:/cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/tensorflow/contrib/tensor_fo

In [15]:
### Trains a combined autoencoder for every merge set
def train_concatamash_autoencoder(histstruct, histslist, vallist, autoencoders):
    
    autoencodersTrain = []
    for i in range(len(histslist)):
        
        print('Now training model {}/'.format(i + 1) + str(len(histslist)))
        
        # Set variables to temporary values for better transparency
        X_train = histslist[i]
        X_val = vallist[i]
        autoencoder = autoencoders[i]
        
        
        ## Model parameters
        nb_epoch = 500
        batch_size = 10000
        
        #checkpoint_filepath = 'checkpoint'
        #model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        #   filepath=checkpoint_filepath,
        #   save_weights_only=False,
        #   verbose=1,
        #   save_best_only=True,
        #   monitor='val_loss',
        #   mode='min')
        
        # Tell the model when to stop
        earlystop = EarlyStopping(monitor='val_loss',
            min_delta=1e-7,
            patience=20,
            verbose=1,
            mode='auto',
            baseline=None,
            restore_best_weights=True,
        )
        lr =0.001
        opt = keras.optimizers.Adam(learning_rate=lr)
        
        autoencoder.compile(loss='mse',
                            optimizer=opt)
        
        ## Train autoencoder
        train = autoencoder.fit(x=[X_train[:,i] for i in range(X_train.shape[1])],
                                y=[X_train[:,i] for i in range(X_train.shape[1])],
                                epochs=nb_epoch,
                                batch_size=batch_size,
                                shuffle=True,
                                validation_data=([X_val[:,i] for i in range(X_val.shape[1])], [X_val[:,i] for i in range(X_val.shape[1])]),
                                verbose=1,
                                callbacks= [earlystop],    
                                )
        tf.keras.utils.plot_model(
                    autoencoder,
                    to_file="models/model1D{}.png".format(i),
                    show_shapes=True,
                    show_dtype=False,
                    show_layer_names=False,
                    rankdir="TB")
        
        # Save classifier for evaluation
        classifier = AutoEncoder.AutoEncoder(model=autoencoder)
        histstruct.add_classifier(histnames[i][0], classifier) 
        autoencodersTrain.append(classifier)
        K.clear_session()
        del(autoencoder, classifier)
    return autoencodersTrain

In [16]:
def load_concatamash_autoencoder():
    for i in range(len(histnames)):
        autoencoder = tf.keras.models.load_model('../SavedModels/Permutations/Job1/AE{}'.format(i))
        classifier = AutoEncoder.AutoEncoder(model=autoencoder)
        histstruct.add_classifier(histnames[i][0], classifier)

In [17]:
start = time.perf_counter()
if trainnew: train_concatamash_autoencoder(histstruct, histslist, vallist, autoencoders)
else: load_concatamash_autoencoder()
stop = time.perf_counter()
print(stop - start)

Now training model 1/11
Epoch 1/500


2022-07-28 19:44:59.940936: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 65280000 exceeds 10% of free system memory.
2022-07-28 19:44:59.972807: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 65280000 exceeds 10% of free system memory.
2022-07-28 19:45:00.554714: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 65280000 exceeds 10% of free system memory.
2022-07-28 19:45:01.170058: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 65280000 exceeds 10% of free system memory.
2022-07-28 19:45:01.170160: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 65280000 exceeds 10% of free system memory.


2/2 [==============================] - 9s 3s/step - loss: 0.0122 - dense_7_loss: 3.5038e-04 - dense_8_loss: 4.2873e-04 - dense_9_loss: 3.8053e-04 - dense_10_loss: 4.0401e-04 - dense_11_loss: 3.2226e-04 - dense_12_loss: 3.9287e-04 - dense_13_loss: 4.0873e-04 - dense_14_loss: 3.5784e-04 - dense_15_loss: 6.1863e-04 - dense_16_loss: 4.8779e-04 - dense_17_loss: 5.3696e-04 - dense_18_loss: 5.2762e-04 - dense_19_loss: 0.0013 - dense_20_loss: 0.0014 - dense_21_loss: 0.0019 - dense_22_loss: 0.0024 - val_loss: 0.0064 - val_dense_7_loss: 2.2580e-04 - val_dense_8_loss: 2.2078e-04 - val_dense_9_loss: 2.0372e-04 - val_dense_10_loss: 2.1014e-04 - val_dense_11_loss: 1.9502e-04 - val_dense_12_loss: 2.2262e-04 - val_dense_13_loss: 2.6642e-04 - val_dense_14_loss: 2.2604e-04 - val_dense_15_loss: 3.5610e-04 - val_dense_16_loss: 2.2763e-04 - val_dense_17_loss: 2.7806e-04 - val_dense_18_loss: 2.8596e-04 - val_dense_19_loss: 6.4221e-04 - val_dense_20_loss: 6.8241e-04 - val_dense_21_loss: 9.9209e-04 - val_dens

Epoch 9/500
2/2 [==============================] - 2s 710ms/step - loss: 3.7186e-04 - dense_7_loss: 2.8131e-05 - dense_8_loss: 2.9989e-05 - dense_9_loss: 2.5890e-05 - dense_10_loss: 2.3357e-05 - dense_11_loss: 2.4792e-05 - dense_12_loss: 2.5976e-05 - dense_13_loss: 2.6072e-05 - dense_14_loss: 2.1538e-05 - dense_15_loss: 2.0199e-05 - dense_16_loss: 1.5817e-05 - dense_17_loss: 2.3534e-05 - dense_18_loss: 1.7473e-05 - dense_19_loss: 3.3254e-05 - dense_20_loss: 1.8839e-05 - dense_21_loss: 1.7555e-05 - dense_22_loss: 1.9443e-05 - val_loss: 3.4444e-04 - val_dense_7_loss: 2.1107e-05 - val_dense_8_loss: 2.6057e-05 - val_dense_9_loss: 2.1455e-05 - val_dense_10_loss: 2.0113e-05 - val_dense_11_loss: 2.0484e-05 - val_dense_12_loss: 2.4170e-05 - val_dense_13_loss: 2.2318e-05 - val_dense_14_loss: 1.6676e-05 - val_dense_15_loss: 1.5082e-05 - val_dense_16_loss: 1.2606e-05 - val_dense_17_loss: 1.7643e-05 - val_dense_18_loss: 1.7128e-05 - val_dense_19_loss: 3.4979e-05 - val_dense_20_loss: 2.1994e-05 - v

Epoch 17/500
2/2 [==============================] - 2s 768ms/step - loss: 1.3858e-04 - dense_7_loss: 1.0893e-05 - dense_8_loss: 1.5178e-05 - dense_9_loss: 1.2509e-05 - dense_10_loss: 7.5679e-06 - dense_11_loss: 8.2200e-06 - dense_12_loss: 1.3360e-05 - dense_13_loss: 1.2869e-05 - dense_14_loss: 5.8443e-06 - dense_15_loss: 4.8281e-06 - dense_16_loss: 6.5840e-06 - dense_17_loss: 6.0119e-06 - dense_18_loss: 5.7018e-06 - dense_19_loss: 1.9749e-05 - dense_20_loss: 2.7970e-06 - dense_21_loss: 3.3851e-06 - dense_22_loss: 3.0788e-06 - val_loss: 1.2715e-04 - val_dense_7_loss: 1.0333e-05 - val_dense_8_loss: 1.4424e-05 - val_dense_9_loss: 1.2104e-05 - val_dense_10_loss: 6.3374e-06 - val_dense_11_loss: 7.6517e-06 - val_dense_12_loss: 1.2780e-05 - val_dense_13_loss: 1.1382e-05 - val_dense_14_loss: 5.1238e-06 - val_dense_15_loss: 3.7932e-06 - val_dense_16_loss: 5.3746e-06 - val_dense_17_loss: 5.7641e-06 - val_dense_18_loss: 5.1425e-06 - val_dense_19_loss: 1.8604e-05 - val_dense_20_loss: 2.6367e-06 - 

...

Epoch 81/500
2/2 [==============================] - 2s 786ms/step - loss: 2.0356e-05 - dense_7_loss: 2.2029e-06 - dense_8_loss: 9.3994e-07 - dense_9_loss: 8.2349e-07 - dense_10_loss: 7.6022e-07 - dense_11_loss: 1.3245e-06 - dense_12_loss: 9.1935e-07 - dense_13_loss: 8.5280e-07 - dense_14_loss: 7.4190e-07 - dense_15_loss: 1.5357e-06 - dense_16_loss: 8.1521e-07 - dense_17_loss: 2.2096e-06 - dense_18_loss: 3.0865e-06 - dense_19_loss: 2.4275e-06 - dense_20_loss: 6.7881e-07 - dense_21_loss: 5.4713e-07 - dense_22_loss: 4.9009e-07 - val_loss: 2.0371e-05 - val_dense_7_loss: 2.1404e-06 - val_dense_8_loss: 9.4336e-07 - val_dense_9_loss: 7.9744e-07 - val_dense_10_loss: 8.0555e-07 - val_dense_11_loss: 1.3146e-06 - val_dense_12_loss: 8.5435e-07 - val_dense_13_loss: 9.0441e-07 - val_dense_14_loss: 7.6237e-07 - val_dense_15_loss: 1.2077e-06 - val_dense_16_loss: 7.1118e-07 - val_dense_17_loss: 2.3306e-06 - val_dense_18_loss: 3.0514e-06 - val_dense_19_loss: 2.4182e-06 - val_dense_20_loss: 8.1208e-07 - 

Epoch 89/500
2/2 [==============================] - 2s 776ms/step - loss: 3.9800e-05 - dense_7_loss: 3.6603e-06 - dense_8_loss: 4.1040e-06 - dense_9_loss: 3.2455e-06 - dense_10_loss: 1.8319e-06 - dense_11_loss: 2.3063e-06 - dense_12_loss: 3.6037e-06 - dense_13_loss: 3.1328e-06 - dense_14_loss: 1.4239e-06 - dense_15_loss: 1.5657e-06 - dense_16_loss: 1.6407e-06 - dense_17_loss: 2.5573e-06 - dense_18_loss: 3.1801e-06 - dense_19_loss: 5.5147e-06 - dense_20_loss: 7.7290e-07 - dense_21_loss: 6.6983e-07 - dense_22_loss: 5.9054e-07 - val_loss: 4.0729e-05 - val_dense_7_loss: 3.5367e-06 - val_dense_8_loss: 4.1953e-06 - val_dense_9_loss: 3.3334e-06 - val_dense_10_loss: 1.9132e-06 - val_dense_11_loss: 2.2738e-06 - val_dense_12_loss: 3.5693e-06 - val_dense_13_loss: 3.2446e-06 - val_dense_14_loss: 1.4787e-06 - val_dense_15_loss: 1.2525e-06 - val_dense_16_loss: 1.5623e-06 - val_dense_17_loss: 2.7166e-06 - val_dense_18_loss: 3.1620e-06 - val_dense_19_loss: 5.9216e-06 - val_dense_20_loss: 9.4336e-07 - 

Epoch 97/500
2/2 [==============================] - 2s 704ms/step - loss: 2.0443e-05 - dense_7_loss: 2.0924e-06 - dense_8_loss: 1.0801e-06 - dense_9_loss: 9.3969e-07 - dense_10_loss: 8.1685e-07 - dense_11_loss: 1.2628e-06 - dense_12_loss: 1.0376e-06 - dense_13_loss: 9.6139e-07 - dense_14_loss: 7.8088e-07 - dense_15_loss: 1.5001e-06 - dense_16_loss: 8.6640e-07 - dense_17_loss: 2.2402e-06 - dense_18_loss: 3.0554e-06 - dense_19_loss: 2.0756e-06 - dense_20_loss: 6.7634e-07 - dense_21_loss: 5.5028e-07 - dense_22_loss: 5.0666e-07 - val_loss: 2.1477e-05 - val_dense_7_loss: 2.3450e-06 - val_dense_8_loss: 1.1474e-06 - val_dense_9_loss: 9.3701e-07 - val_dense_10_loss: 8.4889e-07 - val_dense_11_loss: 1.4131e-06 - val_dense_12_loss: 1.0222e-06 - val_dense_13_loss: 1.0312e-06 - val_dense_14_loss: 7.9265e-07 - val_dense_15_loss: 1.1609e-06 - val_dense_16_loss: 7.2253e-07 - val_dense_17_loss: 2.2696e-06 - val_dense_18_loss: 2.9820e-06 - val_dense_19_loss: 2.7836e-06 - val_dense_20_loss: 7.8796e-07 - 

Epoch 105/500


KeyboardInterrupt: 

In [ ]:
### Evaluate the Models for WP definition
def evaluate_models_train(histstruct):
    
    for histgroup in histnames:
        print('evaluating model for '+histgroup[0])
        print(histstruct.evaluate_classifier(histgroup)[0].shape)
    
    # get mse for training set
    if 'training' in histstruct.masks.keys(): masknames = ['dcson','highstat', 'training']
    else: masknames = ['dcson','highstat']
    mse_train = histstruct.get_scores_array( masknames=masknames )
    print('Found mse array for training set of following shape: {}'.format(mse_train.shape))
    
    # get mse for good set
    if 'good' in histstruct.masks.keys():
        mse_good = []
        for histname in histstruct.histnames:
            mse_good.append(histstruct.get_scores( histname=histname, masknames=['dcson','highstat','good'] ))
    else:
        mse_good = []
        for histname in histstruct.histnames:
            hists_good = hu.averagehists( histstruct.get_histograms( histname=histname, masknames=['dcson','highstat']), 1000 )
            thismse = histstruct.classifiers[histname].evaluate( hists_good )
            mse_good.append( thismse )
            print(run)
    mse_good = np.array(mse_good)
    mse_good = np.transpose(mse_good)
    print('Found mse array for good set of following shape: {}'.format(mse_good.shape))
    
    # get mse for bad sets
    mse_bad = []
    for i in range(nbadruns):
        mse_bad.append( histstruct.get_scores_array( masknames=['dcson','highstat','bad{}'.format(i)] ) )
        print('Found mse array for bad set of following shape: {}'.format(mse_bad[i].shape))
        
    return [mse_train, mse_good, mse_bad]

In [ ]:
(mse_train, mse_good_eval, mse_bad_eval) = evaluate_models_train(histstruct)

In [ ]:
### Plots and Distribution Analysis
def fit_mse_distribution(histstruct, mse_train):
    dimslist = []
    fitfunclist = []
    
    
    nhisttypes = len(histstruct.histnames)
    for i in range(0,nhisttypes-1):
        for j in range(i+1,nhisttypes):
            dimslist.append((i, j))
    
    plt.close('all')
    (npoints,ndims) = mse_train.shape
    
    
    # settings for GaussianKdeFitter
    scott_bw = npoints**(-1./(ndims+4))
    bw_method = 20*scott_bw
    # settings for HyperRectangleFitter
    quantiles = ([0.00062,0.0006,0.00015,0.00015,
                 0.0003,0.0003,0.00053,0.00065])
    
    
    #for dims in dimslist:
    #    thismse = mse_train[:,dims]
    #    if training_mode=='global': 
    #        fitfunc = SeminormalFitter.SeminormalFitter(thismse)
    #        #fitfunc = HyperRectangleFitter.HyperRectangleFitter(thismse, 
    #        #                                                    [quantiles[dims[0]],quantiles[dims[1]]],
    #        #                                                    'up')
    #    else: fitfunc = GaussianKdeFitter.GaussianKdeFitter(thismse,bw_method=bw_method)
    #    #pu.plot_fit_2d(thismse, fitfunc=fitfunc, logprob=True, clipprob=True,
    #    #                onlycontour=False, xlims=30, ylims=30, 
    #    #                onlypositive=True, transparency=0.5,
    #    #                xaxtitle=histstruct.histnames[dims[0]], 
    #    #                yaxtitle=histstruct.histnames[dims[1]],
    #    #                title='density fit of lumisection MSE')
    #    ##plt.close('all') # release plot memory
    #    fitfunclist.append(fitfunc)
    # 
    #    
    if training_mode=='global': 
        fitfunc = SeminormalFitter.SeminormalFitter(mse_train)
        #fitfunc = HyperRectangleFitter.HyperRectangleFitter(mse_train, quantiles, 'up')
    else: 
        fitfunc = GaussianKdeFitter.GaussianKdeFitter()
        fitfunc.fit(mse_train,bw_method=bw_method)
    
    return fitfunc

In [ ]:
fitfunc = fit_mse_distribution(histstruct, mse_train)

In [ ]:
### Prepare MSEs for Working Point Definition
def mse_analysis(histstruct, mse_good_eval, mse_bad_eval, fitfunc):
    
    # Get the minimum log probability of histograms in good set
    print('--- good lumesections ---')
    logprob_good = np.log(fitfunc.pdf(mse_good_eval))
    print('length of log prob array: '+str(len(logprob_good)))
    print('minimum of log prob: '+str(np.min(logprob_good)))
    #print(sorted(logprob_good))
    
    print('--- bad lumisections ---')
    logprob_bad_parts = [np.log(fitfunc.pdf(mse_bad_eval[j])) for j in range(len(mse_bad_eval))]
    print(mse_bad_eval)
    #for lp in logprob_bad_parts: print(str(sorted(lp))+'\n\n')
    logprob_bad = np.concatenate(tuple(logprob_bad_parts))
    
    print('length of log prob array: '+str(len(logprob_bad)))
    print('maximum of log prob: '+str(np.max(logprob_bad)))
    #print(sorted(logprob_good))
    #print(sorted(logprob_bad))
    #print(logprob_bad)
    
    sep = np.min(logprob_good) - np.max(logprob_bad)
    print('Separability: ' + str(sep))
    
    return [logprob_good, logprob_bad]

In [ ]:
(logprob_good, logprob_bad) = mse_analysis(histstruct, mse_good_eval, mse_bad_eval, fitfunc)

In [ ]:
def evaluate_autoencoders_combined(logprob_good, logprob_bad, fmBiasFactor, wpBiasFactor):
    labels_good = np.zeros(len(logprob_good)) # background: label = 0
    labels_bad = np.ones(len(logprob_bad)) # signal: label = 1
    
    badMin = min(np.where(logprob_bad != -np.inf, logprob_bad, -1))
    goodMax = max(np.where(logprob_good != np.inf, logprob_good, 10001))
    
    logprob_good = np.where(logprob_good != np.inf, logprob_good, goodMax)
    logprob_bad = np.where(logprob_bad != -np.inf, logprob_bad, badMin)
    
    # These only take effect if a histogram is grossly misclassified
    logprob_good[logprob_good == -np.inf] = badMin
    logprob_bad[logprob_bad == np.inf] = goodMax
    
    labels = np.concatenate(tuple([labels_good,labels_bad]))
    scores = np.concatenate(tuple([-logprob_good,-logprob_bad]))
    scores = aeu.clip_scores( scores )
    
    avSep = np.mean(logprob_good) - np.mean(logprob_bad)
    
    print('Average Separation: ' + str(avSep))
    
    pu.plot_score_dist(scores, labels, siglabel='anomalous', sigcolor='r', 
                       bcklabel='good', bckcolor='g', 
                       nbins=200, normalize=True,
                       xaxtitle='negative logarithmic probability',
                       yaxtitle='number of lumisections (normalized)')
      
    # Plot ROC curve for analysis
    auc = aeu.get_roc(scores, labels, mode='geom', doprint=False)
    
    # Setting a threshold, below this working point defines anomalous data
    # Average is biased towards better recall per user specifications
    logprob_threshold = (1/(wpBiasFactor + 1)) * (wpBiasFactor*np.mean(logprob_good) + np.mean(logprob_bad))
    # Or set manual
    # logprob_threshold = 100
    (_, _, _, tp, fp, tn, fn) = aeu.get_confusion_matrix(scores,labels,-logprob_threshold)
    print('Selected logprob threshold of ' + str(logprob_threshold))
    
    # Get metrics for analysis
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f_measure = (1 + fmBiasFactor * fmBiasFactor) * ((precision * recall) / ((fmBiasFactor * fmBiasFactor * precision) + recall)) 
    
    print('Accuracy: ' + str(accuracy))
    print('Precision: ' + str(precision))
    print('Recall: ' + str(recall))
    print('F-Measure: ' + str(f_measure))
    
    return logprob_threshold

In [ ]:
logprob_threshold = evaluate_autoencoders_combined(logprob_good, logprob_bad, fmBiasFactor, wpBiasFactor)